<a href="https://colab.research.google.com/github/eli-osherovich/mnist-demo/blob/main/mnist_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist, fashion_mnist

In [2]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-4ad56268-ed75-26b5-1bfe-a9e896d40486)


In [3]:
# Set training context (TPU, GPU or CPU)
try:
  tpu =  tf.distribute.cluster_resolver.TPUClusterResolver()
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])

  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)

  strategy = tf.distribute.TPUStrategy(tpu)
  # policy = tf.keras.mixed_precision.experimental.Policy('mixed_bfloat16')
except ValueError:
  print("Cannot conect to tpu, using default accelerator")
  strategy = tf.distribute.get_strategy()
  # policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')



Cannot conect to tpu, using default accelerator


In [4]:
tf.config.optimizer.set_jit(True)

In [5]:
# the data, split between train and test sets
dataset = fashion_mnist
dataset = mnist


(x_train, y_train), (x_test, y_test) = dataset.load_data()

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

x_train = x_train.astype(np.float32)/255
x_test = x_test.astype(np.float32)/255
y_train = y_train.astype(np.int)
y_test = y_test.astype(np.int)

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

11493376/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
x_test shape: (10000, 28, 28, 1)


In [6]:
def gen_model(activation="relu"):
  s = tf.keras.Input(shape=(28, 28, 1))
  h = tf.keras.layers.Flatten()(s)
  h = tf.keras.layers.Dense(128, activation=activation)(h)
  out = tf.keras.layers.Dense(10)(h)
  return tf.keras.Model(inputs=s, outputs=out)


def compile_model(model, lr=0.001):
  model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=keras.optimizers.Adam(learning_rate=lr),
                metrics=['accuracy'])
  

early_stop = tf.keras.callbacks.EarlyStopping(
      monitor='val_accuracy',
      patience=10,
      restore_best_weights=True,
)

In [7]:
%%time
with strategy.scope():
  model = gen_model()
  compile_model(model)
  model.fit(x_train, y_train,
            batch_size=128,
            epochs=100,
            verbose=1,
            callbacks=[early_stop],
            validation_data=(x_test, y_test),
            validation_batch_size=2048)


Epoch 1/100
469/469 [==============================] - 2s 5ms/step - loss: 0.3637 - accuracy: 0.8987 - val_loss: 0.1954 - val_accuracy: 0.9408
Epoch 2/100
469/469 [==============================] - 2s 5ms/step - loss: 0.1657 - accuracy: 0.9525 - val_loss: 0.1366 - val_accuracy: 0.9613
Epoch 3/100
469/469 [==============================] - 1s 2ms/step - loss: 0.1176 - accuracy: 0.9667 - val_loss: 0.1101 - val_accuracy: 0.9677
Epoch 4/100
469/469 [==============================] - 1s 3ms/step - loss: 0.0900 - accuracy: 0.9739 - val_loss: 0.0959 - val_accuracy: 0.9716
Epoch 5/100
469/469 [==============================] - 1s 3ms/step - loss: 0.0727 - accuracy: 0.9793 - val_loss: 0.0911 - val_accuracy: 0.9726
Epoch 6/100
469/469 [==============================] - 1s 2ms/step - loss: 0.0599 - accuracy: 0.9827 - val_loss: 0.0775 - val_accuracy: 0.9777
Epoch 7/100
469/469 [==============================] - 1s 2ms/step - loss: 0.0500 - accuracy: 0.9857 - val_loss: 0.0810 - val_accuracy: 0.9746

In [8]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Test loss: 0.09318169206380844
Test accuracy: 0.9806000590324402
